In [19]:
# импорт библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


In [20]:

# Шаг 1: Подготовка данных

# Функция для генерации последовательностей x и соответствующих y
def generate_sequences(length):
    x = np.random.randint(0, 10, size=length)  # Генерируем случайную последовательность x
    y = (x + x[0]) % 10  # Рассчитываем соответствующую последовательность y
    return x, y


In [21]:

# Генерация трех датасетов разной длины
dataset_lengths = [25, 75, 150]
train_datasets = []
test_datasets = []

for length in dataset_lengths:
    x_list, y_list = zip(*[generate_sequences(length) for _ in range(1200)])  # Создаем 1200 примеров для каждой длины
    x_array = np.array(x_list, dtype=np.float32)
    x_tensor = torch.tensor(x_array, dtype=torch.float32)
    y_tensor = torch.tensor(y_list, dtype=torch.float32)
    
    # Проверяем, что x и y имеют одинаковое количество элементов
    assert x_tensor.size(0) == y_tensor.size(0), "Size mismatch between x and y"
    
    # Разделение на обучающую и тестовую выборки
    train_x = x_tensor[:100]
    train_y = y_tensor[:100]
    test_x = x_tensor[100:]
    test_y = y_tensor[100:]
    
    train_dataset = TensorDataset(train_x, train_y)
    test_dataset = TensorDataset(test_x, test_y)
    
    train_datasets.append(train_dataset)
    test_datasets.append(test_dataset)


In [26]:

# Шаг 2: Определение архитектур моделей

class MyRNN(nn.Module):
    def __init__(self):
        super(MyRNN, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 25)
    def forward(self, x):
        out, _ = self.rnn(x)
        print(f"RNN output sixe: {out.size()}")
        out = out[:, -1, :]  # Берем только последний выходной вектор из последовательности
        out = self.fc(out)
        print(f"FC output size: {out.size()}")
        return out


class MyLSTM(nn.Module):
    def __init__(self):
        super(MyLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 25)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Берем только последний выходной вектор из последовательности
        out = self.fc(out)
        return out

class MyGRU(nn.Module):
    def __init__(self):
        super(MyGRU, self).__init__()
        self.gru = nn.GRU(input_size=1, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 25)

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]  # Берем только последний выходной вектор из последовательности
        out = self.fc(out)
        return out


In [27]:

# Шаг 3: Обучение моделей

# Функция для обучения модели
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for x, y in train_loader:
            optimizer.zero_grad()

            # Изменение размерности x перед передачей в модель
            x = x.view(x.size(0), -1)
            print(f"x size: {x.size()}")  # Печать размерности входных данных

            outputs = model(x.unsqueeze(2))
            outputs = outputs.view(-1, outputs.size(-1))
            print(f"outputs size: {outputs.size()}")  # Печать размерности выходных данных
            print(f"y size: {y.size()}")  # Печать размерности целевого тензора

            loss = criterion(outputs, y.long().view(-1, 1))
            loss.backward()
            optimizer.step()


In [28]:

# Шаг 4: Оценка производительности моделей

# Создаем заголовок для таблички с результатами
print(f"{'Модель':<10}{'Длина 25':<15}{'Длина 75':<15}{'Длина 150':<15}")

# Цикл по архитектурам моделей
for model_type, ModelClass in [("RNN", MyRNN), ("LSTM", MyLSTM), ("GRU", MyGRU)]:
    row = f"{model_type:<10}"
    
    # Цикл по датасетам
    for i, dataset in enumerate(train_datasets):
        train_loader = DataLoader(train_datasets[i], batch_size=64, shuffle=True) # где i-индекс текущегшо датасета
        model = ModelClass()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        train_model(model, train_loader, criterion, optimizer)
        
        # Оценка производительности на тестовых данных
        test_loader = DataLoader(test_datasets[i], batch_size=64, shuffle=False)
        test_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for x, y in test_loader:
                print(f"x size: {x.size()}")
                outputs = model(x.unsqueeze(2))
                outputs = outputs.view(-1, outputs.size(-1))
                print(f"outputs size: {outputs.size()}")  # Печать размерности выходных данных
                print(f"y size: {y.size()}")
                loss = criterion(outputs.view(-1, outputs.size(-1)),y.long().reshape(-1))
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()
        accuracy = 100 * correct / total
        row += f"{test_loss/len(test_loader):<15.4f}{accuracy:<15.2f}"
    print(row)


Модель    Длина 25       Длина 75       Длина 150      
x size: torch.Size([64, 25])
RNN output sixe: torch.Size([64, 25, 64])
FC output size: torch.Size([64, 25])
outputs size: torch.Size([64, 25])
y size: torch.Size([64, 25])


ValueError: Expected input batch_size (64) to match target batch_size (1600).